Este dois exercícios são apresentados como introdução à modelação de problemas através de “SMT solvers” (Z3, CVC5,…) via uma das interfaces Python  (z3-solver  ou pySMT ) .

# Exercício 1

Neste exercício vai-se considerar a modelação de circuitos do 2º grau com falhas como estão descritos no documento +Trabalho prático: circuito com falhas . Recomenda-se a leitura atenta desse documento antes de considerar o resto do enunciado deste exercício.

Construa uma resolução das seguintes questões a partir de “inputs” do problema: os parâmetros $\,\kappa\,,\,n\,$ e de probabilidade de falha $\,\varepsilon\,$ restrita apenas às “gates” and.


1. Construa algoritmos para, sob “inputs” do segredo $\,z\in\{0,1\}^n\,$ e da “chave mestra” $\,s\in\{0,1\}^\kappa\,$, construa o circuito. Adicionalmente a partir deste circuito , construa o modelo SMT do circuito com falhas.
2. Usando o modelo acima, tente construir uma possível estimativa  para $z$ numa execução com falhas não nulas; isto é, encontrar  $\,z'\in \{0,1\}^n\,$  que é raíz de todos os polinómios que formam o circuito e uma situação de falhas não nulas em “gates” and  que conduz a essa estimativa.
3. Supondo que o circuito não introduz falhas em “gates” mas permite falhas nos “inputs”, conhecido  $\,z\in\{0,1\}^n\,$ pretende-se maximizar a probabilidade de falhas no “input” do circuito sem que o “output” $\,0^n\,$ seja alterado.

$$f(p;x)=o⊕⟨a⋅x⟩⊕(⟨b⋅x⟩×⟨c⋅x⟩)$$

In [2]:
from z3 import *
import numpy as np
from functools import reduce

Precisamos para calcular o $z$ a função sem nenhuma falha. Também definida a função $⟨a⋅b⟩$ $\,a,\,b\in\{0,1\}^n\,$ que usamos na função $f(p;x)$

In [3]:
# funçao f(p;x)

def f_true(p, x):
    o, a, b, c = p

    # produto interno mod 2
    def dot(u, v):
        return int(np.bitwise_xor.reduce(u & v))  # retorna 0 ou 1

    ax = dot(a, x)
    bx = dot(b, x)
    cx = dot(c, x)

    return o ^ ax ^ (bx & cx)






### Funções Geradoras

Defininimos as funções "helpers" que ajudam na criação das variaveis como o $a,b,c$ e é calculado o offset $\,o\in\{0,1\}^n\,$ com base no $\,z\in\{0,1\}^n\,$.

In [4]:
# funçoes de criaçao helpers

def zip_quads(V):
    O,A,B,C = V
    return [(O[i], A[i], B[i], C[i]) for i in range(len(O))]

def make_seed(k):
    bits =  np.random.randint(0, 2, size=k).tolist()
    return int("".join(str(b) for b in bits), 2)

def make_secret(n):
    return np.random.randint(0, 2, size=n).tolist()

def make_locals_s(seed,n):
    rng = np.random.default_rng(seed)
    seeds = rng.integers(0, 2**32, size=n, dtype=np.uint32)
    return seeds

def gen_vector(seeds,n,Z): # gera o meu vetor O
    A, B, C, O = [],[],[],[]
    for seed in range(len(seeds)):
        rng = np.random.default_rng(int(seeds[seed]))
        a = rng.integers(0, 2, size=n, dtype=np.uint8)
        b = rng.integers(0, 2, size=n, dtype=np.uint8)
        c = rng.integers(0, 2, size=n, dtype=np.uint8)

        o = f_true((0,a,b,c), Z)

        A.append(a)
        B.append(b)
        C.append(c)
        O.append(o)

        
        
    #print("O ",O)
    #print("A ",A)
    #print("B ",B)
    #print("C ",C)
    return (O,A,B,C)


### Definição do $z$ e da seed

Aqui Escolhemos o nosso $z$ e o $k$:

In [19]:

k = 4

Z = [1,1,0,0,0,0,0,1,1]
print(Z)
n = len(Z)

seed = make_seed(k) #posso escolher minha seed com base no k 
#seed = 123 # se eu quiser "marcar" uma seed tb posso o fazer

#print(seed)

seeds = make_locals_s(seed,n)
#print(seeds)



vecs = gen_vector(seeds,n,Z)
#print(vecs)

O = vecs[0]

p = zip_quads(vecs)
#print(p)



#terminou o "setup"

[1, 1, 0, 0, 0, 0, 0, 1, 1]


### Definição da chave publica

Com o $o$ e a $s$ temos o par que é a chave publica

In [20]:
q = (O,seed) # com esse q preciso achar o z
print(q)

([0, 0, 0, 0, 1, 0, 1, 0, 1], 12)


### Funções Auxiliares pro Solver

- $regen\_params(seed,n)$ : retorna o grupo de vetores $A,B,C$ (tamanho $n$) de acordo com a $seed$ escolhida anteriormente.
- $dots(a,z)$ ou $⋅$ : retorna a logica da definição desta função que é feita no enunciado mas retorna numa variavel $BoolVal$ aceite pelo o solver $Z_3$



In [21]:
def regen_params(seed, n):
    A, B, C = [], [], []
    seeds = make_locals_s(seed,n)
    for seed in range(len(seeds)):
        rng = np.random.default_rng(int(seeds[seed]))  # mesma inicialização
        a = rng.integers(0, 2, size=n, dtype=np.uint8)
        b = rng.integers(0, 2, size=n, dtype=np.uint8)
        c = rng.integers(0, 2, size=n, dtype=np.uint8)
        A.append(a)
        B.append(b)
        C.append(c)
    return A, B, C

def dot(a, z):
    return reduce(lambda x, y: Xor(x, y),
                  [z[j] for j in range(len(a)) if a[j]],
                  BoolVal(False))



### Função de Decriptação Teorica (sem falhas)

A resolução da função $decrypt\_theoric(q,n)$ foi feita para uma visualização mais facil do exercicio 

In [22]:
def decrypt_theoric(q, n):
    O, seed = q
    A, B, C = regen_params(seed, n)

    #print("O ", O)
    #print("A ", A)
    #print("B ", B)
    #print("C ", C)

    Z = [Bool(f"z_{i}") for i in range(n)]
    
    s = Solver()

    for i in range(n):
        a, b, c, o = A[i], B[i], C[i], O[i]




        offset = BoolVal(o == 1)
        dots = Xor(dot(a, Z), And(dot(b, Z), dot(c, Z)))  # isto devia ter uma chance de erro

        s.add(Not(Xor(offset, dots)))

    if s.check() == sat:
        model = s.model()
        z_val = [model[zj] for zj in Z]
        print(''.join(['1' if model[zj] else '0' for zj in Z]))
        return [bool(model[zj]) for zj in Z]
    else:
        print("nenhuma soluçao encontrada")
        return None


decrypt_theoric(q, n)


    

    

011110000


[False, True, True, True, True, False, False, False, False]

Facilmente foi feita a solução de encontrar o $z$ adicionando no Solver as restrições para cada indice $i$ nos vetores $O_i,A_i,B_i,C_i$ com $i\in \{0..n\}$


### Resolução da Função de Decriptação com Falhas (maximizando a probabilidade de falha)

A resolução deste exercicio é parecida com a resolução da função passada, basta adicionarmos uma variavel solver a mais que é a $E_i,  i \in \{0..n*red\}$ Sendo $red$ um parametro novo que nos diz quantas "redundancias" o circuito faz, ou seja quantas vezes nos repetimos o ponto critico do circuito para fazer o *majority* (exemplo nas aulas teoricas sobre circuitos). $E$ Podera so ter 2 valores, reforçados pela restrição: $\forall i \in \{0..n*red\}  E_i = 0 \vee E_i = 1$ adicionando tambem o contexto/restrição:
- $E_i = 0 $  *sse*  O $w_i$ não tem falha
- $E_i = 1 $  *sse*  O $w_i$ tem falha

Para $\forall i \in \{0..n*red\}$ e para $w_i \in$ todos os wires do circuito ($n*red$ wires no total)

In [23]:
def decrypt_real(q, n,red): #agora adicionando redundancia
    O, seed = q
    A, B, C = regen_params(seed, n)

    #print("O ", O)
   # print("A ", A)
    #print("B ", B)
    #print("C ", C)

    Z = [Bool(f"z_{i}") for i in range(n)]
    E = [(Int(f"d_{i}")) for i in range(red*n)] # 0: nao existe falha; 1: existe falha

    d_vars = [d for d in E]  # pega só os d_i
    
    s = Optimize()

    for d in E:
        s.add(Or(d == 0, d == 1))

    s.add(Sum(E) > 0)

    contas_valids = []
    
    for i in range(n):
        
        a, b, c, o = A[i], B[i], C[i], O[i]

        offset = BoolVal(o == 1)
        
        red_v = []
        and_v = If(And(dot(b, Z), dot(c, Z)),1,0) # calculo sem erro do and para nao ter que refazer o calculo red vezes 
        
        for j in range(red):
            d_j = E[i*red + j]
            v = If(d_j == 0   , and_v,IntVal(-1) ) #adiciona -1 no valor do fio caso falhe
            red_v.append(v)
            

        count_valid = Sum([If(Or(v == 0, v == 1), 1, 0) for v in red_v]) #contando os validos 
        contas_valids.append(count_valid)
        
        s.add(count_valid > Sum([If(v == -1, 1, 0) for v in red_v])) # aqui é a restrição do majority
        

        dots = Xor(dot(a, Z), And(dot(b, Z), dot(c, Z))) 

        s.add(Not(Xor(offset, dots)))
        

    total = Sum(contas_valids)
    
    
    s.minimize(total)

    if s.check() == sat:
        model = s.model()
        z_val = [is_true(model[zj]) for zj in Z]
        print("Z =", ''.join(['1' if val else '0' for val in z_val]))
    
        wires_on = []
        wires_fail = []
    
        for i, (d) in enumerate(E):
            d_val = model.evaluate(d, model_completion=True).as_long()  
    
            if d_val == 1:   
                wires_fail.append(i)
            else:            
                wires_on.append(i)
    
        total_wires = len(E)
        fail_ratio = len(wires_fail) / total_wires * 100
    
        print(f"\n=== Status dos wires ===")
        print(f"Funciona ({len(wires_on)}): {wires_on}")
        print(f"Falha ({len(wires_fail)}): {wires_fail}")
        print(f"Probabiliade de falha: {fail_ratio:.2f}%\n")
    
        return z_val
    else:
        print("Nenhuma solução encontrada")
        return None


decrypt_real(q, n,7)

Z = 010110111

=== Status dos wires ===
Funciona (36): [1, 2, 3, 6, 7, 8, 10, 11, 14, 17, 19, 20, 21, 23, 24, 26, 29, 31, 32, 33, 35, 37, 38, 41, 42, 43, 45, 46, 51, 53, 54, 55, 57, 59, 60, 62]
Falha (27): [0, 4, 5, 9, 12, 13, 15, 16, 18, 22, 25, 27, 28, 30, 34, 36, 39, 40, 44, 47, 48, 49, 50, 52, 56, 58, 61]
Probabiliade de falha: 42.86%



[False, True, False, True, True, False, True, True, True]

Para estimar a probabilidade de falha é feito o calculo $(F/T)*100$ com 
- $F$: nº de fios com falha
- $T$: nº total de fios no circuito.
  
E o nosso objetivo é maximizar a probabilidade de falha ou seja maximizar o $F$ ou dualmente Minimizar o $T-F$ e foi o que foi feito na função $decrypt\_real(q, n,red)$  